# Part 1: Building the MVP AI System  

In this first part, we’ll build a **zero-shot classification system** using a local **LLM-powered approach**. Instead of training a model from scratch, we’ll leverage **Gemma 2B**’s in-context learning to classify sports teams as **from the US or Australia**—without any fine-tuning.  

But an AI system is **more than just a model**. We'll also integrate:  
- **Gradio** to build an interactive front end.  
- **SQLite** to store data and results.  
- **Datasette** for observability, allowing us to inspect predictions and iterate effectively.  

By the end of this section, you’ll have a **working MVP AI system**—a functional app with a front end, database, and structured observability to track and refine performance.  
